Loading data, importing libraries, and importing utils

In [1]:
from typing import *
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from models import SimpleModel
import torchvision.transforms.v2  as F
from torchviz import make_dot
import tqdm

from sklearn.svm import LinearSVC
from main import train,test,extract_features,train_with_patches,test_patches,extract_patch_features
from fisher_vector import neural_based_fisher
from IPython.display import clear_output
from torch.utils.data import TensorDataset, DataLoader

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"

os.environ["CUDA_VISIBLE_DEVICES"]="0"

os.environ['JOBLIB_TEMP_FOLDER'] = '/home/msiau/workspace/jventosa/PostTFG/Master/project-1/Week2/joblib'


/home/msiau/workspace/jventosa/miniconda3/envs/C3_Team1/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)


#### MLP on small images

Entire Training

In [3]:
def train_simple_model(model,model_name,train_loader,test_loader):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 100

    train_losses, train_accuracies = [], []
    test_losses, test_accuracies = [], []

    best_test_loss = 10_000_000
    train_loss_at_best = 10_000_000
    best_test_accuracy =10_000_000
    best_epoch = 0
    for epoch in tqdm.tqdm(range(num_epochs), desc="TRAINING THE MODEL"):
        train_loss, train_accuracy = train(model, train_loader, criterion, optimizer, device)
        test_loss, test_accuracy = test(model, test_loader, criterion, device)

        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        test_losses.append(test_loss)
        test_accuracies.append(test_accuracy)

        if test_loss < best_test_loss:
            best_test_loss = test_loss
            train_loss_at_best = train_accuracy
            best_test_accuracy = test_accuracy
            best_epoch = epoch+1
            torch.save(model.state_dict(), f"SimpleModel/{model_name}.pth")

        print(f"Epoch {epoch + 1}/{num_epochs} - "
                f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
                f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
    return best_test_accuracy, train_loss_at_best, best_epoch

Finding optimal Image Size

In [4]:
image_search_results = {}

for image_size in [224,14,28,56,112]:
    
    
    train_transformation = F.Compose([
    F.ToImage(),
    F.ToDtype(torch.float32, scale=True),
    F.Resize(size=(image_size, image_size)),  # Slightly larger for cropping
    F.RandomResizedCrop(size=(image_size, image_size)),
    F.RandomHorizontalFlip(p=0.2),
    F.RandomRotation(degrees=15),
    ])
    
    transformation  = F.Compose([
                                    F.ToImage(),
                                    F.ToDtype(torch.float32, scale=True),
                                    F.Resize(size=(image_size, image_size)),
                                ])

    data_train = ImageFolder("../places_reduced/train", )
    data_test = ImageFolder("../places_reduced/val", transform=transformation) 


    train_images = []
    train_labels = []
    for img, label in data_train:
        train_images.append(img)
        train_labels.append(label)

    train_images = torch.stack(train_images).to(device=device)
    train_labels = torch.tensor(train_labels,device=device)

    print("Loading test data to VRAM...")
    test_images = []
    test_labels = []
    for img, label in data_test:
        test_images.append(img)
        test_labels.append(label)

    test_images = torch.stack(test_images).to(device=device)
    test_labels = torch.tensor(test_labels,device=device)

    # Create TensorDatasets with data already on GPU
    train_dataset_gpu = TensorDataset(train_images, train_labels,transform=train_transformation)
    test_dataset_gpu = TensorDataset(test_images, test_labels)

    train_loader = DataLoader(train_dataset_gpu, batch_size=256,  shuffle=True, num_workers=0)
    test_loader = DataLoader(test_dataset_gpu, batch_size=128,  shuffle=False, num_workers=0)
    
    C, H, W = np.array(data_train[0][0]).shape

    input_size = C*H*W

    hidden_layers_n = 2

    hidden_dim = 300

    model = SimpleModel(input_d=C*H*W,hidden_layers_n=hidden_layers_n, hidden_d=hidden_dim, output_d=11)

    model_name = f"{input_size}_input_{hidden_layers_n}_layers_{hidden_dim}_dimension"
    
    image_search_results[input_size] = train_simple_model(model,model_name,train_loader,test_loader)
    




Loading test data to VRAM...


TRAINING THE MODEL:   1%|          | 1/100 [00:03<05:10,  3.14s/it]

Epoch 1/100 - Train Loss: 4.0966, Train Accuracy: 0.1123, Test Loss: 2.3848, Test Accuracy: 0.1482


TRAINING THE MODEL:   2%|▏         | 2/100 [00:04<03:31,  2.15s/it]

Epoch 2/100 - Train Loss: 2.3369, Train Accuracy: 0.1482, Test Loss: 2.2599, Test Accuracy: 0.1659


...

Epoch 98/100 - Train Loss: 0.0323, Train Accuracy: 0.9964, Test Loss: 9.5931, Test Accuracy: 0.2009


TRAINING THE MODEL:  99%|█████████▉| 99/100 [00:25<00:00,  4.37it/s]

Epoch 99/100 - Train Loss: 0.0302, Train Accuracy: 0.9967, Test Loss: 9.5846, Test Accuracy: 0.1932


TRAINING THE MODEL: 100%|██████████| 100/100 [00:25<00:00,  3.86it/s]

Epoch 100/100 - Train Loss: 0.0212, Train Accuracy: 0.9987, Test Loss: 9.7614, Test Accuracy: 0.1977


In [10]:
print(image_search_results)

{150528: (0.24272727272727274, 0.2754022988505747, 7), 588: (0.2781818181818182, 0.3120689655172414, 19), 2352: (0.2781818181818182, 0.3249425287356322, 18), 9408: (0.28136363636363637, 0.30896551724137933, 12), 37632: (0.26636363636363636, 0.3082758620689655, 11)}


Finding optimal number of layers

In [11]:
optimal_image_size = 112
transformation  = F.Compose([
                                F.ToImage(),
                                F.ToDtype(torch.float32, scale=True),
                                F.Resize(size=(optimal_image_size, optimal_image_size)),
                            ])

data_train = ImageFolder("../places_reduced/train", transform=transformation)
data_test = ImageFolder("../places_reduced/val", transform=transformation) 

train_loader = DataLoader(data_train, batch_size=256, pin_memory=True, shuffle=True, num_workers=8)
test_loader = DataLoader(data_test, batch_size=128, pin_memory=True, shuffle=False, num_workers=8)

In [15]:
layer_search_results = {}

for hidden_layers_n in [1,2,3,4,5,6,7,8]:
    
    C, H, W = np.array(data_train[0][0]).shape

    input_size = C*H*W


    hidden_dim = 300

    model = SimpleModel(input_d=C*H*W,hidden_layers_n=hidden_layers_n, hidden_d=hidden_dim, output_d=11)

    model_name = f"{input_size}_input_{hidden_layers_n}_layers_{hidden_dim}_dimension"
    
    layer_search_results[hidden_layers_n] = train_simple_model(model,model_name,train_loader,test_loader)
    

TRAINING THE MODEL:   1%|          | 1/100 [00:06<11:23,  6.90s/it]

Epoch 1/100 - Train Loss: 3.1389, Train Accuracy: 0.1329, Test Loss: 2.2877, Test Accuracy: 0.1627


TRAINING THE MODEL:   2%|▏         | 2/100 [00:13<11:14,  6.88s/it]

Epoch 2/100 - Train Loss: 2.2311, Train Accuracy: 0.1905, Test Loss: 2.1764, Test Accuracy: 0.2150


TRAINING THE MODEL:   3%|▎         | 3/100 [00:20<11:04,  6.85s/it]

...

Epoch 98/100 - Train Loss: 1.1233, Train Accuracy: 0.6072, Test Loss: 3.1534, Test Accuracy: 0.2441


TRAINING THE MODEL:  99%|█████████▉| 99/100 [10:49<00:06,  6.56s/it]

Epoch 99/100 - Train Loss: 1.0816, Train Accuracy: 0.6209, Test Loss: 3.4712, Test Accuracy: 0.2359


TRAINING THE MODEL: 100%|██████████| 100/100 [10:56<00:00,  6.56s/it]

Epoch 100/100 - Train Loss: 1.0454, Train Accuracy: 0.6333, Test Loss: 3.7020, Test Accuracy: 0.2282


In [16]:
print(layer_search_results)

{1: (0.2640909090909091, 0.30344827586206896, 10), 2: (0.25181818181818183, 0.3032183908045977, 11), 3: (0.2731818181818182, 0.31666666666666665, 12), 4: (0.26590909090909093, 0.29954022988505746, 14), 5: (0.25181818181818183, 0.27632183908045976, 12), 6: (0.24727272727272728, 0.2944827586206897, 23), 7: (0.24227272727272728, 0.29022988505747127, 25), 8: (0.23545454545454544, 0.2671264367816092, 21)}


Finding the optimal layer dimensions

In [ ]:
dimension_search_results = {}

optimal_layer_n = 3

for hidden_dim in [50,100,150,200,250,300,350,400,450,500]:
    
    C, H, W = np.array(data_train[0][0]).shape

    input_size = C*H*W



    model = SimpleModel(input_d=C*H*W,hidden_layers_n=optimal_layer_n, hidden_d=hidden_dim, output_d=11)

    model_name = f"{input_size}_input_{hidden_layers_n}_layers_{hidden_dim}_dimension"
    
    dimension_search_results[hidden_dim] = train_simple_model(model,model_name,train_loader,test_loader)
    

TRAINING THE MODEL:   1%|          | 1/100 [00:06<11:03,  6.70s/it]

Epoch 1/100 - Train Loss: 2.3328, Train Accuracy: 0.1439, Test Loss: 2.2584, Test Accuracy: 0.1673


TRAINING THE MODEL:   2%|▏         | 2/100 [00:13<10:47,  6.61s/it]

Epoch 2/100 - Train Loss: 2.2336, Train Accuracy: 0.1895, Test Loss: 2.2064, Test Accuracy: 0.1750


TRAINING THE MODEL:   3%|▎         | 3/100 [00:19<10:41,  6.61s/it]

...

TRAINING THE MODEL:  99%|█████████▉| 99/100 [10:47<00:06,  6.55s/it]

Epoch 99/100 - Train Loss: 0.0012, Train Accuracy: 1.0000, Test Loss: 9.9545, Test Accuracy: 0.2318


TRAINING THE MODEL: 100%|██████████| 100/100 [10:53<00:00,  6.54s/it]

Epoch 100/100 - Train Loss: 0.0012, Train Accuracy: 1.0000, Test Loss: 10.0299, Test Accuracy: 0.2336


: 

Loading model and testing

In [11]:
C, H, W = np.array(data_train[0][0]).shape
model = SimpleModel(input_d=C*H*W,hidden_layers_n=2, hidden_d=300, output_d=11)

model = model.to(device)
model.load_state_dict(torch.load("SimpleModel/BestWeights.pth", weights_only=True))

criterion = nn.CrossEntropyLoss()

test_loss, test_accuracy = test(model, test_loader, criterion, device)

print(test_loss,test_accuracy)

2.0375764231248334 0.2631818181818182


#### SVM on layers from model

We will use the same model as last cell, and try some taking features from each layer

In [12]:
hidden_layers_n = 2

for layer_id in range(hidden_layers_n+1):
    train_feats, train_labels = extract_features(model, train_loader, device, layer_id)
    test_feats, test_labels = extract_features(model, test_loader, device, layer_id)

    svm = LinearSVC(C=1.0)
    svm.fit(train_feats, train_labels)
    svm_acc = svm.score(test_feats, test_labels)

    print(f"SVM accuracy using layer {layer_id}: {svm_acc:.4f}")

/home/msiau/workspace/jventosa/miniconda3/envs/C3_Team1/lib/python3.8/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


SVM accuracy using layer 0: 0.1364


/home/msiau/workspace/jventosa/miniconda3/envs/C3_Team1/lib/python3.8/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/msiau/workspace/jventosa/miniconda3/envs/C3_Team1/lib/python3.8/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


SVM accuracy using layer 1: 0.1532


/home/msiau/workspace/jventosa/miniconda3/envs/C3_Team1/lib/python3.8/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/msiau/workspace/jventosa/miniconda3/envs/C3_Team1/lib/python3.8/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


SVM accuracy using layer 2: 0.2614


/home/msiau/workspace/jventosa/miniconda3/envs/C3_Team1/lib/python3.8/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


#### Patch predictions joined

In [13]:
C, H, W = np.array(data_train[0][0]).shape

patch_size = int(H / 4)

hidden_layers_n = 2

model = SimpleModel(input_d=C*patch_size*patch_size,hidden_layers_n=hidden_layers_n, hidden_d=300, output_d=11)


model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 20

train_losses, train_accuracies = [], []
test_losses, test_accuracies = [], []

best_test_loss = 10_000_000

for epoch in tqdm.tqdm(range(num_epochs), desc="TRAINING THE PATCH BASED MODEL"):
    train_loss, train_accuracy = train_with_patches(model, train_loader, criterion, optimizer, device,patch_size)
    test_loss, test_accuracy = test_patches(model, test_loader, criterion, device,patch_size)

    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

    if test_loss < best_test_loss:
        best_test_loss = test_loss
        os.makedirs(f"PatchModel_{patch_size}",exist_ok=True)
        torch.save(model.state_dict(), f"PatchModel_{patch_size}/BestWeights.pth")

    print(f"Epoch {epoch + 1}/{num_epochs} - "
            f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
            f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

TRAINING THE PATCH BASED MODEL:   5%|▌         | 1/20 [00:10<03:21, 10.59s/it]

Epoch 1/20 - Train Loss: 2.3898, Train Accuracy: 0.1157, Test Loss: 2.3190, Test Accuracy: 0.1441


TRAINING THE PATCH BASED MODEL:  10%|█         | 2/20 [00:21<03:09, 10.55s/it]

Epoch 2/20 - Train Loss: 2.3178, Train Accuracy: 0.1489, Test Loss: 2.2692, Test Accuracy: 0.2036


TRAINING THE PATCH BASED MODEL:  15%|█▌        | 3/20 [00:31<03:01, 10.66s/it]

...

Epoch 18/20 - Train Loss: 2.2138, Train Accuracy: 0.2024, Test Loss: 2.1320, Test Accuracy: 0.2518


TRAINING THE PATCH BASED MODEL:  95%|█████████▌| 19/20 [03:22<00:10, 10.60s/it]

Epoch 19/20 - Train Loss: 2.2132, Train Accuracy: 0.2027, Test Loss: 2.1391, Test Accuracy: 0.2595


TRAINING THE PATCH BASED MODEL: 100%|██████████| 20/20 [03:32<00:00, 10.64s/it]

Epoch 20/20 - Train Loss: 2.2105, Train Accuracy: 0.2037, Test Loss: 2.1520, Test Accuracy: 0.2577


In [14]:
C, H, W = np.array(data_train[0][0]).shape

patch_size = int(H / 4)

hidden_layers_n = 2

model = SimpleModel(input_d=C*patch_size*patch_size,hidden_layers_n=hidden_layers_n, hidden_d=300, output_d=11)

model = model.to(device)
model.load_state_dict(torch.load(f"PatchModel_{patch_size}/BestWeights.pth", weights_only=True))

criterion = nn.CrossEntropyLoss()

test_loss, test_accuracy = test_patches(model, test_loader, criterion, device,patch_size)

print(test_loss,test_accuracy)

2.12735369682312 0.2759090909090909


#### Passing patch features to BOVW

Our best method was ___ 

In [15]:


C, H, W = np.array(data_train[0][0]).shape

patch_size = int(H / 4)

hidden_layers_n = 2

model = SimpleModel(input_d=C*patch_size*patch_size,hidden_layers_n=hidden_layers_n, hidden_d=300, output_d=11)

model = model.to(device)
model.load_state_dict(torch.load(f"PatchModel_{patch_size}/BestWeights.pth", weights_only=True))


for layer_id in range(hidden_layers_n+1):
    train_feats, train_labels = extract_patch_features(model, train_loader, device, layer_id,patch_size)
    test_feats, test_labels = extract_patch_features(model, test_loader, device, layer_id,patch_size)
    print(train_feats.shape)
    print(train_labels.shape)
    acc , _ = neural_based_fisher(train_feats, test_feats, train_labels, test_labels)

    print(f"BOVW accuracy using layer {layer_id}: {acc:.4f}")

(8700, 16, 300)
(8700,)
y
Fitting GMM with K=128, cov_type=diag on 2000 descriptors...
Initialization 0
  Iteration 10
  Iteration 20
Initialization converged: True


Computing Fisher Vectors (Parallel): 100%|██████████| 8700/8700 [00:41<00:00, 210.22it/s]


Extracting descriptors for test set...


Computing Fisher Vectors (Parallel): 100%|██████████| 2200/2200 [00:04<00:00, 531.94it/s]


mamaMia
pizzeria


...

Extracting descriptors for test set...


Computing Fisher Vectors (Parallel): 100%|██████████| 2200/2200 [00:03<00:00, 608.69it/s]


mamaMia
pizzeria


/home/msiau/workspace/jventosa/miniconda3/envs/C3_Team1/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fitted
Predicted
BOVW accuracy using layer 2: 0.3077
